# Day 16 - Advent of Code

## Requirement
Imagine having a beam light passing through space containing mirrors and splitters. We need to determine the path of the light. The beam light can move in four direction: Upward (90 degree), Downward (-90 degree), Right (0 degree), Left (180 degree).
We are given a 2D matrix containing three types of characters: empty space (.), mirrors (/ and \), and splitters (| and -).
- empty space (.) allow the light to go in the same direction
- / and \ reflect the light 90 degre, depending of the direction:
  - Upward (90) + / --> Right (0)
  - Upward (90) + \ --> Left (180)
  - Downward (-90) + / --> Left (-180)
  - Downward (-90) + \ --> Right (0)
  - Left (180) + / --> Downward(-90)
  - Left (180) + \ --> Upward(90)
  - Right (0) + / --> Upward (90)
  - Right (0) + \ --> Downward (-90)
- If the light enounter the pointy end of the spliter (| or -), it will pass through the splitter.
  - Example: a 0 degree or 180 degree beam light will pass through -, a 90 and -90 beam light will pass through |
- If the light encouter the flat side of the spliter (| or -), it will split into half:
  - If the beam moving left or right encounter |, it will split into two beams: one moving upward, one moving downward
  - If the beam moving up or down encounter -, it will split into two beams: one moving left, one moving right

We count the energized tile, which has at least one beam pass through it, reflect in it, split in it. 
In other word, we count the number of visited tile.

## Algorithm
- Define a mapping from a beam direction, and the matrix symbol to the next direction. 
- Have a queue to consider the beams. The queue start with the first beam, and we add new beams to the queue.
- We track the path of each beam, and count the tiles visited by these tracks

Read the data from the input file

In [22]:
from collections import deque
input_path = "day16.txt"
data = []
with open(input_path, "r") as f:
    lines = f.readlines()
    for line in lines:
        line = line.replace("\n", "").strip()   
        if line:     
            data.append(list(line))

ROW, COL = len(data), len(data[0])
print(ROW, COL)

110 110


Define the mapping

In [23]:
mapping = {
    "left": {
        "/": ["down"],
        "\\": ["up"],
        "-": ["left"],
        ".": ["left"],
        "|": ["up", "down"]
    },
    "right": {
        "/": ["up"],
        "\\": ["down"],
        "-": ["right"],
        ".": ["right"],
        "|": ["up", "down"]

    },
    "up": {
        "/": ["right"],
        "\\": ["left"],
        "-": ["left", "right"],
        ".": ["up"],
        "|": ["up"]
    },
    "down": {
        "/": ["left"],
        "\\": ["right"],
        "-": ["left", "right"],
        ".": ["down"],
        "|": ["down"]
    },
}
direction_coord = {
    "left": [0, -1],
    "right": [0, 1],
    "up": [-1, 0],
    "down": [1, 0],
}

Start the the first position, moving right, and consider the next direction based on the rule defined by the problem.

We need to keep track of the visited node position and the direction, so that we don't visit a node with the same direction twice

In [24]:
def count_genergized_tile(start=(0, 0), direction="right"):
    visited = {}
    queue = deque([(start, direction)])
    while queue:
        (row, col), current_direction = queue.popleft()
        if ((row, col), current_direction) in visited:
            continue

        visited[((row, col), current_direction)] = True
        for next_direction in mapping[current_direction][data[row][col]]:
            new_row = row + direction_coord[next_direction][0]
            new_col = col + direction_coord[next_direction][1]
            if 0 <= new_row < ROW and 0 <= new_col < COL:
                queue.append(((new_row, new_col), next_direction))
    return len(set([key[0] for key in visited.keys()]))
print(count_genergized_tile())

6816


Time complexity: O(MxNxK), where M is the number of rows, N is the number of columns, and K is the number of directions

Space compleixty: O(MxNxK)

## Second problem
- Now we can start at any position on the top, bottom, left, right of the matrix
- Find the position with the maximum number of tile engergized. We get the maximum engergized tiles
- We can use the same aglorithm for in the first problem for this problem, with the time complexity O(max(M,N)xNxMxK). This is still doable with the given input O(110x110x110x110x4) ~ O(10^8)

In [25]:
ans = 0
for row in range(ROW):
    ans = max(ans, count_genergized_tile((row, 0), "right"))
    ans = max(ans, count_genergized_tile((row, COL - 1), "left"))

for col in range(COL):
    ans = max(ans, count_genergized_tile((0, col), "down"))
    ans = max(ans, count_genergized_tile((ROW - 1, col), "up"))
print(ans)

8163


Can we optimize this algorithm further?
Yes! There are only MxNxK possible combinations of position and direction. 
If the beam position and direction is in the cached => we don't need to consider it again. 
We maintain the maximum number of engerized tiles for each visitedn position. 
We only update the new position and direction


20121
